In [1]:
%matplotlib inline
import matplotlib as mlp
mlp.rcParams['figure.figsize'] = 10, 15
import matplotlib.pyplot as plt
import rawpy
import numpy as np

In [2]:
#load with no input in postprocess
raw = rawpy.imread("./hq_cam_color_checker_new.dng")

In [3]:
array_color = raw.postprocess()

In [8]:
array = raw.postprocess(output_color=rawpy.ColorSpace.raw, gamma=(1,1),
                        user_wb=[1.0,1.0,1.0,1.0], no_auto_bright=True)

array_crop = array[2300:2800, 2050:2550] # grey field #D4
# field 4 last line [y1:y2,x1:x2] 500x500
r_mean = np.mean(array_crop[:,:,0])
g_mean = np.mean(array_crop[:,:,1])
b_mean = np.mean(array_crop[:,:,2])
base_wb = [g_mean/r_mean, 1.0, g_mean/b_mean, 1.0]

array_corr = raw.postprocess(user_wb=base_wb)

In [4]:
import os
import csv

def write_data(data, path):
    '''this function takes a bunch of data as a list-object
    and writes those to a csv file'''
    file = "dng_cc_color_rgb_dist.csv"
    file = os.path.join(path, file)
    with open(file, 'a+', newline='') as new_row:
        csv_writer = csv.writer(new_row)
        csv_writer.writerow(data)


In [5]:
# A1 - F4
dict_cc_colors = {"A1":[115,82,68],
                "A2":[214,126,44],
                "A3":[56,61,150],
                "A4":[243,243,242],
                "B1":[194,150,130],
                "B2":[80,91,166],
                "B3":[70,148,73],
                "B4":[200,200,200],
                "C1":[98,122,157],
                "C2":[193,90,99],
                "C3":[175,54,60],
                "C4":[160,160,160],
                "D1":[87,108,67],
                "D2":[94,60,108],
                "D3":[231,199,31],
                "D4":[122,122,121],
                "E1":[133,128,177],
                "E2":[157,188,64],
                "E3":[187,86,149],
                "E4":[85,85,85],
                "F1":[103,189,170],
                "F2":[224,126,44],
                "F3":[8,133,161],
                "F4":[52,52,52]
                }


array_cc_colors = [["A1", "B1", "C1", "D1", "E1", "F1"],
                    ["A2", "B2", "C2", "D2", "E2", "F2"],
                    ["A3", "B3", "C3", "D3", "E3", "F3"],
                    ["A4", "B4", "C4", "D4", "E4", "F4"]]



# A1 - F4
dict_cc_field = {"A1":[250,750, 50,550],
                "A2":[950,1450, 50,550],
                "A3":[1600,2100, 50,550],
                "A4":[2300,2800, 50,550],
                "B1":[250,750, 750,1250],
                "B2":[950,1450, 750,1250],
                "B3":[1600,2100, 750,1250],
                "B4":[2300,2800, 750,1250],
                "C1":[250,750, 1400,1900],
                "C2":[950,1450, 1400,1900],
                "C3":[1600,2100, 1400,1900],
                "C4":[2300,2800, 1400,1900],
                "D1":[250,750, 2050,2550],
                "D2":[950,1450, 2050,2550],
                "D3":[1600,2100, 2050,2550],
                "D4":[2300,2800, 2050,2550],
                "E1":[250,750, 2750,3250],
                "E2":[950,1450, 2750,3250],
                "E3":[1600,2100, 2750,3250],
                "E4":[2300,2800, 2750,3250],
                "F1":[250,750, 3450,3950],
                "F2":[950,1450, 3450,3950],
                "F3":[1600,2100, 3450,3950],
                "F4":[2300,2800, 3450,3950]
                }

In [9]:
# loop for colors in color checker

postprocess_info = ["corrected"]

for y in range(4):
    for x in range(6):
        # links nach rechts, oben nach unten
        # crop erst x anpassen, dann y
        y1 = int(dict_cc_field[array_cc_colors[y][x]][0])
        y2 = int(dict_cc_field[array_cc_colors[y][x]][1])
        x1 = int(dict_cc_field[array_cc_colors[y][x]][2])
        x2 = int(dict_cc_field[array_cc_colors[y][x]][3])

        array_crop = array_corr[y1:y2,x1:x2]
        # calculate mean values for the cropped field
        r_mean = np.mean(array_crop[:,:,0])
        g_mean = np.mean(array_crop[:,:,1])
        b_mean = np.mean(array_crop[:,:,2])

        r_dist = dict_cc_colors[array_cc_colors[y][x]][0] - r_mean
        g_dist = dict_cc_colors[array_cc_colors[y][x]][1] - g_mean 
        b_dist = dict_cc_colors[array_cc_colors[y][x]][2] - b_mean
        
        distance = np.sqrt(r_dist**2+g_dist**2+b_dist**2)

        data = [dict_cc_colors[array_cc_colors[y][x]], r_dist, g_dist, b_dist, distance, postprocess_info]
        write_data(data, "./")
